In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")

client = OpenAI(api_key=api_key)

### STT (Speech To Text)

In [2]:
file_path = "C://Users//fursew//project//open_ai//ch05//음성녹음.mp3"

with open(file_path,"rb") as f:
    response = client.audio.transcriptions.create(
        file=f,
        model="whisper-1",
    )

response.text

'다시 말해봐 일단 내가 2천원 먼저 보내줄테니까 12,000원 나중에 보내줄게 왜 또 줄어? 가격이? 그만 그렇게 말하는거지 너 왔다며 아니 그냥 10,000원 보내주고 5,000원 나중에 대 12,000원 2,000원 하자 12,000원 2,000원 아니 1,000원은 뭐 그렇게 아쉬워 야 그럼 12,000원 12,000원 그냥 12,000원 두고 3,000원 그냥 아쉬워 나는 지금 많이 희생했잖아 나는 3,000원 내 버스비긴 해 내 교통비라서 미치겠네 아쉬워라 3,000원이 그걸 나중에 내면 된다니까 지금 내라는게 아니냐 몇번을 말해 언제 내가 보려면 계속 두사람 기다려야되는데 그건 니가 그냥 일을 안하는거네 돈없다고 찡찡대면서 돈은 아니잖아 일단 머리 자르고 돈은 보내주던 연락을 할게 있는 돈부터 계산을 해봐야 될거 아니야 막냉이 얼마인지 계산 안될 수 있잖아 일단 뭐 뭐가 됐든 일단 이따 말해 그래 업보가 되겠네 엥? 업보는 아니고'

In [4]:
# translation을 통한 영어 번역
with open(file_path,"rb") as f:
    response = client.audio.translations.create(
    file = f,
    model = "whisper-1"
)

response.text

"I'll cut my hair. Say that again. I'll send you 2,000 won first. I'll send you 12,000 won later. Why is the price going down? Stop talking like that. You said you were here. No, I'll just send you 10,000 won. I'll pay you 5,000 won later. Let's do 12,000 won, 2,000 won. 12,000 won, 2,000 won. 1,000 won is not enough. Then 12,000 won. You're not gonna give me 3,000 won? No, I've sacrificed a lot right now. I'll pay you 3,000 won later. It's my bus fare. It's my transportation fee. I'm sorry. I'll pay you 3,000 won later. I'll pay you 3,000 won later. I'm not asking you to pay me now. How many times do I have to tell you? When? I have to wait two more months to see you. You're just not working. You're complaining that you don't have money. I don't have money. I'll cut my hair first. I'll send you the money. I'll call you later. You have to pay me first. I don't know how much you have. I can't pay you. Whatever it is, I'll tell you later. Yeah, I'll give you a kiss. What?"

### 로컬에서 사용하기

In [ ]:
os.environ["PATH"] += os.pathsep + "C://PMPG//ffmpeg-2025-07-23-git-829680f96a-full_build//bin"

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=5,
    stride_length_s=1,
)

sample = "C://Users//fursew//project//open_ai//ch05//음성녹음.mp3"

result = pipe(sample)
print(result)

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/2

{'text': ' 다시 말해봐 일단 내가 이천원 먼저 보내줄테니깐 1만 2천원 나중에 보내줄게 왜 또 줄어 가격이 그만 그렇게 말하면 하지 너 왔다며 아니 그냥 만원 보내주고 오천원 나중에 돼 12,000원 12,000원 12,000원 아니 천 원이 하자 12,000원에 2,000원 아니 천원이 뭐 그 자조 저건 많이 터러 12,000원은 그냥 12,000원에 3해 너도 삼촌은 그게 아쉬워? 아니 나는 지금 많이 희생했잖아 나는 3천은 내 버스비긴 해가지고 내 교통비라서 아쉬워놔 3천원이 그걸 나중에 내면 된다니까 지금 내라는 게 아니냐 몇 번을 말해? 언제? 한 보려면 계속 두 달은 기다려야 되는데 그건 니가 그냥 일을 안 하는 거네 돈 없다고 찡찡대면서 일단 머리 자르고 돈을 보내주던 연락을 할게 있는 돈부터 계산을 해봐야 될 거 아니야 남는 돈이 얼마인지 계산 안될 수 있잖아 일단 뭐 뭐가 됐든 이따 말해 그래, 오뽀가 되겠네 엇? 엇보는 아니고', 'chunks': [{'timestamp': (0.0, 2.0), 'text': ' 다시 말해봐'}, {'timestamp': (2.0, 5.0), 'text': ' 일단 내가 이천원 먼저 보내줄테니깐'}, {'timestamp': (5.0, 9.08), 'text': ' 1만 2천원 나중에 보내줄게 왜 또 줄어 가격이'}, {'timestamp': (9.08, 10.54), 'text': ' 그만 그렇게 말하면 하지'}, {'timestamp': (10.54, 11.66), 'text': ' 너 왔다며'}, {'timestamp': (11.66, 14.0), 'text': ' 아니 그냥 만원 보내주고'}, {'timestamp': (14.0, 17.7), 'text': ' 오천원 나중에 돼 12,000원'}, {'timestamp': (17.7, 18.02), 'text': ' 12,000원'}, {'timestamp': (18.02, 18.72), 'text': ' 12,000원'}, {'timestam

In [7]:
df_list = []
for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    df_list.append([start,end,text])

import pandas as pd

df = pd.DataFrame(df_list,columns=["start","end","text"])
df

,start,end,text
0,0.00,2.00,다시 말해봐
1,2.00,5.00,일단 내가 이천원 먼저 보내줄테니깐
2,5.00,9.08,1만 2천원 나중에 보내줄게 왜 또 줄어 가격이
3,9.08,10.54,그만 그렇게 말하면 하지
4,10.54,11.66,너 왔다며
5,11.66,14.00,아니 그냥 만원 보내주고
6,14.00,17.70,"오천원 나중에 돼 12,000원"
7,17.70,18.02,"12,000원"
8,18.02,18.72,"12,000원"
9,18.72,19.30,"아니 천 원이 하자 12,000원에 2,000원"


In [1]:
# instantiate the pipeline
from dotenv import load_dotenv
import os
import torch
from pyannote.audio import Pipeline

load_dotenv()
token = os.getenv("Hugging_API_KEY")

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=token)

if torch.cuda.is_available():
    pipeline.to(torch.device("cuda:0"))
    print("cuda is available")
else:
    pipeline.to(torch.device("cpu"))
    print("cuda is not available")

# run the pipeline on an audio file
diarization = pipeline("C://Users//fursew//project//open_ai//ch05//음성녹음.mp3")

# dump the diarization output to disk using RTTM format
with open("audio.rttm", "w",encoding='utf-8') as rttm:
    diarization.write_rttm(rttm)

c:\Users\fursew\project\open_ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\fursew\AppData\Local\Programs\Python\Python311\Lib\inspect.py:992: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


cuda is available


c:\Users\fursew\project\open_ai\.venv\Lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
c:\Users\fursew\project\open_ai\.venv\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
c:\Users\fursew\project\open_ai\.venv\Lib\site-packages\torchaudio\_backend\soundfile_backend.py:71: UserWarning: The MP

- start
- duration
- speaker_id

In [ ]:
import pandas as pd

df = pd.read_csv("C://Users//fursew//project//open_ai//ch05//audio.rttm",
names=["type","file_name","chnl","start","duration","C1","C2",
"speaker_id","C3","C4"],sep=" ",header=None)
df.head()

,type,file_name,chnl,start,duration,C1,C2,speaker_id,C3,C4
0,SPEAKER,음성녹음,1,0.031,1.164,NaN,NaN,SPEAKER_00,NaN,NaN
1,SPEAKER,음성녹음,1,2.748,1.586,NaN,NaN,SPEAKER_01,NaN,NaN
2,SPEAKER,음성녹음,1,4.739,1.097,NaN,NaN,SPEAKER_01,NaN,NaN
3,SPEAKER,음성녹음,1,7.270,1.451,NaN,NaN,SPEAKER_00,NaN,NaN
4,SPEAKER,음성녹음,1,9.143,0.253,NaN,NaN,SPEAKER_01,NaN,NaN


In [5]:
df['end'] = df['start'] + df['duration']
display(df)

,type,file_name,chnl,start,duration,C1,C2,speaker_id,C3,C4,end
0,SPEAKER,음성녹음,1,0.031,1.164,NaN,NaN,SPEAKER_00,NaN,NaN,1.195
1,SPEAKER,음성녹음,1,2.748,1.586,NaN,NaN,SPEAKER_01,NaN,NaN,4.334
2,SPEAKER,음성녹음,1,4.739,1.097,NaN,NaN,SPEAKER_01,NaN,NaN,5.836
3,SPEAKER,음성녹음,1,7.270,1.451,NaN,NaN,SPEAKER_00,NaN,NaN,8.721
4,SPEAKER,음성녹음,1,9.143,0.253,NaN,NaN,SPEAKER_01,NaN,NaN,9.396
5,SPEAKER,음성녹음,1,9.717,1.974,NaN,NaN,SPEAKER_01,NaN,NaN,11.691
6,SPEAKER,음성녹음,1,10.308,0.253,NaN,NaN,SPEAKER_00,NaN,NaN,10.561
7,SPEAKER,음성녹음,1,12.367,0.506,NaN,NaN,SPEAKER_00,NaN,NaN,12.873
8,SPEAKER,음성녹음,1,13.042,1.215,NaN,NaN,SPEAKER_00,NaN,NaN,14.257
9,SPEAKER,음성녹음,1,14.662,0.894,NaN,NaN,SPEAKER_00,NaN,NaN,15.556


In [6]:
# 화자 발언 관련 컬럼 만들기
df['num_spk'] = None
df.iloc[0,-1] = 0
df.head()

,type,file_name,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,num_spk
0,SPEAKER,음성녹음,1,0.031,1.164,NaN,NaN,SPEAKER_00,NaN,NaN,1.195,0
1,SPEAKER,음성녹음,1,2.748,1.586,NaN,NaN,SPEAKER_01,NaN,NaN,4.334,None
2,SPEAKER,음성녹음,1,4.739,1.097,NaN,NaN,SPEAKER_01,NaN,NaN,5.836,None
3,SPEAKER,음성녹음,1,7.270,1.451,NaN,NaN,SPEAKER_00,NaN,NaN,8.721,None
4,SPEAKER,음성녹음,1,9.143,0.253,NaN,NaN,SPEAKER_01,NaN,NaN,9.396,None


In [ ]:
df['num_spk'] = None
df.iloc[0,-1] = 0
for i in range(1,len(df)):
    if df['speaker_id'].iloc[i] != df['speaker_id'].iloc[i-1]:
        df['num_spk'].iloc[i] = df['num_spk'].iloc[i-1] + 1
    else:
        df['num_spk'].iloc[i] = df['num_spk'].iloc[i-1]
df.head()

C:\Users\fursew\AppData\Local\Temp\ipykernel_17244\3911745501.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['num_spk'].iloc[i] = df['num_spk'].iloc[i-1] + 1
C:\Users\fursew\AppData\Local\Temp\ipykernel_17244\3911745501.py:3: Setting

,type,file_name,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,num_spk
0,SPEAKER,음성녹음,1,0.031,1.164,NaN,NaN,SPEAKER_00,NaN,NaN,1.195,0
1,SPEAKER,음성녹음,1,2.748,1.586,NaN,NaN,SPEAKER_01,NaN,NaN,4.334,1
2,SPEAKER,음성녹음,1,4.739,1.097,NaN,NaN,SPEAKER_01,NaN,NaN,5.836,1
3,SPEAKER,음성녹음,1,7.270,1.451,NaN,NaN,SPEAKER_00,NaN,NaN,8.721,2
4,SPEAKER,음성녹음,1,9.143,0.253,NaN,NaN,SPEAKER_01,NaN,NaN,9.396,3


In [8]:
df_grouped = df.groupby('num_spk').agg(
    start = pd.NamedAgg(column='start', aggfunc='min'),
    end = pd.NamedAgg(column='end', aggfunc='max'),
    speaker_id = pd.NamedAgg(column='speaker_id', aggfunc='first')
)
df_grouped

,start,end,speaker_id
num_spk,,,
0,0.031,1.195,SPEAKER_00
1,2.748,5.836,SPEAKER_01
2,7.270,8.721,SPEAKER_00
3,9.143,11.691,SPEAKER_01
4,10.308,15.556,SPEAKER_00
5,16.012,26.541,SPEAKER_01
6,27.537,30.254,SPEAKER_00
7,30.372,34.591,SPEAKER_01
8,34.591,34.777,SPEAKER_00


In [9]:
df_grouped['duration'] = df_grouped['end'] - df_grouped['start']
df_grouped.reset_index(drop=True,inplace=True)
df_grouped.head()

,start,end,speaker_id,duration
0,0.031,1.195,SPEAKER_00,1.164
1,2.748,5.836,SPEAKER_01,3.088
2,7.270,8.721,SPEAKER_00,1.451
3,9.143,11.691,SPEAKER_01,2.548
4,10.308,15.556,SPEAKER_00,5.248


In [10]:
df_grouped.to_csv("음성녹음_rttm.csv",index=False)